<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/Bert_adultreadmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install keras_bert

In [5]:
import json
import numpy as np
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os, gc
import codecs
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import roc_auc_score


Using TensorFlow backend.


In [0]:
maxlen = 300
config_path = 'uncased_L-12_H-768_A-12/bert_config.json'
checkpoint_path = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'uncased_L-12_H-768_A-12/vocab.txt'

In [0]:
token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [0]:
class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            # elif self._is_space(c):
            #     R.append('[unused1]')
            else:
                R.append('[UNK]')
        return R

tokenizer = OurTokenizer(token_dict)

In [0]:
data = pd.read_csv('drive/My Drive/Adult readmission/Combined_vars_04092019_notes.csv')
data = data[['opnote_1', 'CODE_REHOSP']].dropna()
data.CODE_REHOSP = np.where(data.CODE_REHOSP==2, 0, 1)
random_order = list(range(len(data)))
np.random.shuffle(random_order)
train_data = [data.iloc[j] for i, j in enumerate(random_order) if i % 5 != 0]
valid_data = [data.iloc[j] for i, j in enumerate(random_order) if i % 5 == 0]

In [0]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

class data_generator:
    def __init__(self, data, batch_size=32):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
          idxs = list(range(len(self.data)))
          np.random.shuffle(idxs)
          X1, X2, Y = [], [], []
          for i in idxs:
              d = self.data[i]
              text = d[0][:maxlen]
              x1, x2 = tokenizer.encode(first=text)
              y = d[1]
              X1.append(x1)
              X2.append(x2)
              Y.append([y])
              if len(X1) == self.batch_size or i == idxs[-1]:
                  X1 = seq_padding(X1)
                  X2 = seq_padding(X2)
                  Y = seq_padding(Y)
                  yield [X1, X2], Y
                  [X1, X2, Y] = [], [], []

In [11]:
bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path)
for l in bert_model.layers:
    l.trainable = True

x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(1, activation='sigmoid')(x)

model = Model([x1_in, x2_in], p)
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-5),
    metrics=[auroc]
)

W0826 18:52:51.124451 140342009886592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 18:52:51.175183 140342009886592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 18:52:51.223665 140342009886592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 18:52:51.224750 140342009886592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0826 18:52:51.234937

In [12]:
train_D = data_generator(train_data)
valid_D = data_generator(valid_data)
model.fit_generator(
    train_D.__iter__(),
    steps_per_epoch=len(train_D),
    epochs=5,
    validation_data=valid_D.__iter__(),
    validation_steps=len(valid_D)
)

Epoch 1/5


ResourceExhaustedError: ignored

In [0]:
del bert_model
gc.collect()